#01 - Data Loading & Initial Exploration

In this notebook, I:
- load the customer support ticket dataset
- inspect structure, missing values, and basic distributions
- create simple text-length features
- identify which columns will be used for later text analytics and modeling 

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
pd.set_option('display.max_colwidth', 200)

# Load Data
df = pd.read_csv('customer_support_tickets.csv')

df.head()


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchased}. Please assist.\n\nYour billing zip code is: 71701.\n\nWe appreciate that you have requested a website address.\n\nPlease double check your email a...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchased}. Please assist.\n\nIf you need to change an existing product.\n\nI'm having an issue with the {product_purchased}. Please assist.\n\nIf The issue I...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,"I'm facing a problem with my {product_purchased}. The {product_purchased} is not turning on. It was working fine until yesterday, but now it doesn't respond.\n\n1.8.3 I really I'm using the origin...",Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,"I'm having an issue with the {product_purchased}. Please assist.\n\nIf you have a problem you're interested in and I'd love to see this happen, please check out the Feedback. I've already contacte...",Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchased}. Please assist.\n\n\nNote: The seller is not responsible for any damages arising out of the delivery of the battleground game. Please have the game...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ticket ID                     8469 non-null   int64  
 1   Customer Name                 8469 non-null   object 
 2   Customer Email                8469 non-null   object 
 3   Customer Age                  8469 non-null   int64  
 4   Customer Gender               8469 non-null   object 
 5   Product Purchased             8469 non-null   object 
 6   Date of Purchase              8469 non-null   object 
 7   Ticket Type                   8469 non-null   object 
 8   Ticket Subject                8469 non-null   object 
 9   Ticket Description            8469 non-null   object 
 10  Ticket Status                 8469 non-null   object 
 11  Resolution                    2769 non-null   object 
 12  Ticket Priority               8469 non-null   object 
 13  Tic

In [6]:
df.isna().sum()

Ticket ID                          0
Customer Name                      0
Customer Email                     0
Customer Age                       0
Customer Gender                    0
Product Purchased                  0
Date of Purchase                   0
Ticket Type                        0
Ticket Subject                     0
Ticket Description                 0
Ticket Status                      0
Resolution                      5700
Ticket Priority                    0
Ticket Channel                     0
First Response Time             2819
Time to Resolution              5700
Customer Satisfaction Rating    5700
dtype: int64

## Dataset Structure
- We have **8,469** tickets and **17** original columns.
- Key columns include:
  - `Ticket Description` (main free-text field)
  - `Ticket Type`, `Product Purchased`, `Ticket Priority`, `Ticket Channel`
  - `Ticket Status` (Open, Closed, Pending Customer Response)
  - `Customer Satisfaction Rating` (1–5, but present only for closed tickets)
- `Resolution`, `Timeto Resolution`, and `Customer Satisfaction Rating`
  are only populated for **closed** tickets; they are missing for open/pending tickets.
- All other columns are fully populated (no missing values).


In [7]:
cols_to_drop = ['Customer Name', 'Customer Email']
df = df.drop(columns=cols_to_drop)

In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ticket ID                     8469 non-null   int64  
 1   Customer Age                  8469 non-null   int64  
 2   Customer Gender               8469 non-null   object 
 3   Product Purchased             8469 non-null   object 
 4   Date of Purchase              8469 non-null   object 
 5   Ticket Type                   8469 non-null   object 
 6   Ticket Subject                8469 non-null   object 
 7   Ticket Description            8469 non-null   object 
 8   Ticket Status                 8469 non-null   object 
 9   Resolution                    2769 non-null   object 
 10  Ticket Priority               8469 non-null   object 
 11  Ticket Channel                8469 non-null   object 
 12  First Response Time           5650 non-null   object 
 13  Tim

## Dropped PII-like columns

I removed: 
- `Customer Name`
- `Customer Email`

These fields are not useful for analytics and would be a privacy issue in the real-world.
The dataset now has **15 columns**. 

In [9]:
#convert purchase date to datetime
df['Date of Purchase'] = pd.to_datetime(df['Date of Purchase'], errors='coerce')

In [10]:
# simple text-length features
df['desc_word_count'] = df['Ticket Description'].apply(lambda x: len(str(x).split()))
df['desc_char_count'] = df['Ticket Description'].apply(lambda x: len(str(x)))
df['subject_word_count'] = df['Ticket Subject'].apply(lambda x: len(str(x).split()))

df.describe(include='all')

,Ticket ID,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating,desc_word_count,desc_char_count,subject_word_count
count,8469.000000,8469.000000,8469,8469,8469,8469,8469,8469,8469,2769,8469,8469,5650,2769,2769.000000,8469.000000,8469.000000,8469.0
unique,NaN,NaN,3,42,NaN,5,16,8077,3,2769,4,4,5470,2728,NaN,NaN,NaN,NaN
top,NaN,NaN,Male,Canon EOS,NaN,Refund request,Refund request,I'm having an issue with the {product_purchased}. Please assist. This problem started occurring after the recent software update. I haven't made any other changes to the device.,Pending Customer Response,We seat culture plan.,Medium,Email,2023-06-01 20:45:39,2023-06-01 17:14:42,NaN,NaN,NaN,NaN
freq,NaN,NaN,2896,240,NaN,1752,576,25,2881,1,2192,2143,3,3,NaN,NaN,NaN,NaN
mean,4235.000000,44.026804,NaN,NaN,2020-12-30 01:35:13.071201024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.991333,46.467352,289.821939,2.0
min,1.000000,18.000000,NaN,NaN,2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,21.000000,151.000000,2.0
25%,2118.000000,31.000000,NaN,NaN,2020-07-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,43.000000,273.000000,2.0
50%,4235.000000,44.000000,NaN,NaN,2020-12-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,49.000000,298.000000,2.0
75%,6352.000000,57.000000,NaN,NaN,2021-07-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,52.000000,318.000000,2.0
max,8469.000000,70.000000,NaN,NaN,2021-12-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,63.000000,397.000000,2.0


## Text Length Features

I created: 
- `desc_word_count`: number of words in each ticket description
- `desc_char_count`: number of characters in each ticket description
- `subject_word_count`: number of words in each ticket subject

From the summary:
- Ticket descriptions average around **46-47** words with a fairly tight range (about 21-63 words).
- This means descriptions are fairly detailed and consistent in length.
- Ticket subjects are usually very short (about **2 words** on average). 

In [11]:
for col in ['Ticket Type', 
            'Product Purchased', 
            'Ticket Priority', 
            'Ticket Channel', 
            'Ticket Status']:
    print(f"\nValue counts for {col}:\n")
    print(df[col].value_counts())


Value counts for Ticket Type:

Ticket Type
Refund request          1752
Technical issue         1747
Cancellation request    1695
Product inquiry         1641
Billing inquiry         1634
Name: count, dtype: int64

Value counts for Product Purchased:

Product Purchased
Canon EOS                         240
GoPro Hero                        228
Nest Thermostat                   225
Amazon Echo                       221
Philips Hue Lights                221
LG Smart TV                       219
Sony Xperia                       217
Roomba Robot Vacuum               216
Apple AirPods                     213
LG OLED                           213
iPhone                            212
Sony 4K HDR TV                    210
LG Washing Machine                208
Garmin Forerunner                 208
Canon DSLR Camera                 206
Nikon D                           204
Nintendo Switch Pro Controller    203
Google Pixel                      203
Fitbit Charge                     202
Sony Pl

In [12]:
df['Date of Purchase'] = pd.to_datetime(df['Date of Purchase'], errors='coerce')
df['First Response Time'] = pd.to_numeric(df['First Response Time'], errors='coerce')
df['Time to Resolution'] = pd.to_numeric(df['Time to Resolution'], errors='coerce')

df['desc_word_count'] = df['Ticket Description'].apply(lambda x: len(str(x).split()))
df['desc_char_count'] = df['Ticket Description'].apply(lambda x: len(str(x)))
df['subject_word_count'] = df['Ticket Subject'].apply(lambda x: len(str(x).split()))

df.info()
df[['desc_word_count', 'desc_char_count', 'subject_word_count']].describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Ticket ID                     8469 non-null   int64         
 1   Customer Age                  8469 non-null   int64         
 2   Customer Gender               8469 non-null   object        
 3   Product Purchased             8469 non-null   object        
 4   Date of Purchase              8469 non-null   datetime64[ns]
 5   Ticket Type                   8469 non-null   object        
 6   Ticket Subject                8469 non-null   object        
 7   Ticket Description            8469 non-null   object        
 8   Ticket Status                 8469 non-null   object        
 9   Resolution                    2769 non-null   object        
 10  Ticket Priority               8469 non-null   object        
 11  Ticket Channel                

,desc_word_count,desc_char_count,subject_word_count
count,8469.000000,8469.000000,8469.0
mean,46.467352,289.821939,2.0
std,8.461730,43.593954,0.0
min,21.000000,151.000000,2.0
25%,43.000000,273.000000,2.0
50%,49.000000,298.000000,2.0
75%,52.000000,318.000000,2.0
max,63.000000,397.000000,2.0


In [13]:
df['Customer Satisfaction Rating'].value_counts(dropna=False)

Customer Satisfaction Rating
NaN    5700
3.0     580
1.0     553
2.0     549
5.0     544
4.0     543
Name: count, dtype: int64

## Categorical Distributions

**Ticket Type** 
The dataset includes five well-balanced ticket types:
- Refund request
- Technical issue
- Cancellation request
- Product inquiry 
- Billing inquiry

**Product Purchased**
There are about 40 different consumer-tech products. 
Counts per product are fairly balanced (~180-240 tickets each), which is helpful both for modeling and for selecting a subset of products for image analysis later.

**Ticket Priority** 
Priorities are also well balanced:
- Low, Medium, High, and Critical all have similar counts (~2000 each).
This gives a realistic mix of urgency levels.

**Ticket Channel**
Tickets are submitted through:
- Email, Phone, Social media, and Chat
These channels are again quite evenly distributed (~2000 each).

**Ticket Status**
Tickets fall into three main statuses:
- Open
- Pending Customer Response
- Closed

Only *closed* tickets have a `Customer Satisfaction Rating` text. 

**Customer Satisfaction Rating**
- Around **2,769** tickets (all closed) have a rating from 1-5. 
- Ratings are fairly evenly distributed across 1-5, with a mean close to 3. 
                                                                                      
This subset of closed tickets will be the basis for our satisfaction prediction model.

In [14]:
#create 'closed tickets only' data subset for both predictive models

df_closed = df[df['Customer Satisfaction Rating'].notna()].copy()
df_closed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2769 entries, 2 to 8467
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Ticket ID                     2769 non-null   int64         
 1   Customer Age                  2769 non-null   int64         
 2   Customer Gender               2769 non-null   object        
 3   Product Purchased             2769 non-null   object        
 4   Date of Purchase              2769 non-null   datetime64[ns]
 5   Ticket Type                   2769 non-null   object        
 6   Ticket Subject                2769 non-null   object        
 7   Ticket Description            2769 non-null   object        
 8   Ticket Status                 2769 non-null   object        
 9   Resolution                    2769 non-null   object        
 10  Ticket Priority               2769 non-null   object        
 11  Ticket Channel                2769 

In [15]:
df_closed['Ticket Status'] = df_closed['Ticket Status'].astype('category')

cat_cols = ['Customer Gender', 'Product Purchased', 'Ticket Type',
            'Ticket Priority', 'Ticket Channel', 'Ticket Status']

for col in cat_cols:
    df_closed[col] = df_closed[col].astype('category')

In [16]:
df_closed[['Time to Resolution', 'Customer Satisfaction Rating']].describe()

,Time to Resolution,Customer Satisfaction Rating
count,0.0,2769.000000
mean,NaN,2.991333
std,NaN,1.407016
min,NaN,1.000000
25%,NaN,2.000000
50%,NaN,3.000000
75%,NaN,4.000000
max,NaN,5.000000


## Modeling Subset: Closed Tickets with Ratings

For modeling, we focus on tickets that:
- Are **Closed**
- Have a non-missing `Customer Satisfaction Rating`

This gives us about **2,769** tickets.  
We will use this `df_model` dataframe in the next notebook for:
- Text cleaning and lemmatization
- Sentiment analysis
- TF-IDF feature creation
- Predicting customer satisfaction.

In [19]:
# Saved the cleaned modeling dataset for next notebooks
df_closed.to_csv('df_model.csv', index=False)
print(f"Saved df_model.csv with {len(df_closed)} closed tickets")

# Also saved with text features for later use
df_model_features = df_closed.copy()
df_model_features.to_csv('df_model_features.csv', index=False)
print(f"Saved df_model_features.csv with {len(df_model_features)} tickets and {len(df_model_features.columns)} columns")

Saved df_model.csv with 2769 closed tickets
Saved df_model_features.csv with 2769 tickets and 18 columns
